## Evaluate trained model


In [9]:
import torch
from concrete_vae.vae import VAE

state_dict = torch.load("../model.pt")
model = VAE(x_dim=784, z_dim=10).load_state_dict(state_dict)

TypeError: __init__() got an unexpected keyword argument 'x_dim'